# Análise Gráfica

&ensp;A análise gráfica é essencial na análise de dados, pois permite visualizar padrões, tendências e anomalias de maneira intuitiva, tornando dados complexos mais acessíveis e compreensíveis. Ao utilizar gráficos, é possível identificar rapidamente observações valiosas, como tendências de crescimento ou declínio, facilitando a comparação entre diferentes conjuntos de dados e a identificação de outliers.

Para mais informações de como rodar, veja o arquivo `orientacao.md`.

* Importação das bibliotecas necessárias:

In [1]:
import pandas as pd # pandas
import numpy as np # numpy
import matplotlib.pyplot as plt  # matplotlib
import seaborn as sns  # seaborn

* Leitura e visualização do _DataFrame_:

In [ ]:
df = pd.read_excel('./data.xlsx') # leitura do arquivo
df.head() # visualização do Data Frame

## Gráficos

### 10 serviços mais utilizados no plano de saúde

In [ ]:
frequencia = df['Descricao Servico Sinistro'].value_counts()  # frequência dos serviços.
top_10 = frequencia.nlargest(10)  # seleciona os 5 serviços mais usados.
moda = df['Descricao Servico Sinistro'].mode()[0]  # moda dos serviços.

colors = ['#0000FF', '#006400', '#800080', '#FFA500', '#00FF00', '#FF0000', '#FF00FF', '#8B4513', '#FFC0CB', '#2F4F4F', '#00FFFF', '#FFFF00']

# gráfico de barras.
plt.figure(figsize=(20, 6))
bars = plt.bar(range(len(top_10)), top_10.values, color=colors)

# rótulos dos serviços para a legenda.
labels = [f'{service} ({count} ocorrências)' for service, count in zip(top_10.index, top_10.values)]

plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])  # Remove os nomes dos serviços do eixo x.
plt.xlabel('Serviços')
plt.ylabel('Quantidade')
plt.title('10 serviços mais utilizados no plano de saúde')

# Adiciona a legenda.
plt.legend(bars, labels)

plt.show()

&ensp;A análise inicial concentrou-se nos dez serviços mais utilizados no plano de saúde e na frequência com que são registrados no banco de dados. Essa análise permitiu a identificação das áreas mais demandadas e de possíveis riscos. Embora fosse esperado que o número de consultas fosse predominante, observou-se um elevado número de sessões de psicoterapia por psicólogos, assim como exames de TSH. Posteriormente, será investigada a razão para o alto volume de exames relacionados ao hormônio tireoestimulante.

### Valor total pago pelos 5 serviços mais utilizados no plano de saúde

In [ ]:
frequencia_servico = df['Descricao Servico Sinistro'].value_counts() # frequência das descrições

top_5 = frequencia_servico.nlargest(5).index # top 5 maiores

df_top_5 = df[df['Descricao Servico Sinistro'].isin(top_5)]

valor_total = df_top_5.groupby('Descricao Servico Sinistro')['Valor Pago Sinistro'].sum().reindex(top_5) # valor total pago por serviço
colors = ['#0000FF', '#006400', '#800080', '#FFA500', '#00FF00']

plt.figure(figsize=(12, 6))

sns.barplot(x=range(len(valor_total)), y=valor_total.values, palette=colors, hue=valor_total.index, dodge=False)

plt.xlabel('Serviços')
plt.ylabel('Valor Total Pago em Milhões')
plt.title('Valor Total Pago para os Top 5 Serviços Mais Utilizados')

plt.show()


A segunda análise concentrou-se no valor total desembolsado pelos cinco serviços mais utilizados no plano de saúde. Essa avaliação permite identificar os serviços mais onerosos e pode fornecer subsídios para o desenvolvimento de programas de saúde direcionados. Observou-se um gasto elevado com sessões de psicoterapia individual realizadas por psicólogos (aproximadamente R$ 575.495,95), o que pode oferecer insights para a criação de programas voltados à saúde mental.

### Gastos por faixa etária e elegibilidade

In [ ]:
# filtro de elegibilidade
dependentes = df[df['Elegibilidade Sinistro'] == 'DEPENDENTE']
nao_dependentes = df[df['Elegibilidade Sinistro'] == 'TITULAR']

df = df.sort_values(by='Faixa-Etária Nova Sinistro', ascending=True) # ordem crescente

# gráfico de disperção
plt.figure(figsize=(20, 6))
plt.scatter(df['Faixa-Etária Nova Sinistro'], df['Valor Pago Sinistro'], color='blue', alpha=0.6)

plt.scatter(dependentes['Faixa-Etária Nova Sinistro'], dependentes['Valor Pago Sinistro'], color='blue', alpha=0.6, label='Dependentes')

plt.scatter(nao_dependentes['Faixa-Etária Nova Sinistro'], nao_dependentes['Valor Pago Sinistro'], color='green', alpha=0.6, label='Não Dependentes')

plt.legend()

plt.xlabel('Idade')
plt.ylabel('Gastos')
plt.title('Dispersão de Gastos por Idade')
plt.grid(True)
plt.show()

&emsp;Tmabém foi elaborado um gráfico para analisar os gastos de acordo com a faixa etária e o tipo de elegibilidade (dependente ou titular). Verificou-se uma maior tendência para serviços mais custosos na faixa dos 49 aos 53 anos, tanto entre dependentes (cônjuges) quanto titulares. Além disso, observou-se que quase todos os registros na faixa de 0 a 18 anos partem de dependentes, o que reflete o uso do plano de saúde para os filhos dos colaboradores. Esses dados podem fornecer uma base sólida para a formulação de hipóteses futuras.

### Quantidade de Sinistros por Mês ao Longo do Tempo

In [ ]:
# Criar uma nova coluna para os bimestres
df['Bimestre'] = (df['Dt Data Sinistro'].dt.month - 1) // 2 + 1
df['Ano_Bimestre'] = df['Dt Data Sinistro'].dt.year.astype(str) + '-B' + df['Bimestre'].astype(str)

# Agrupar os dados pelos bimestres e contar os sinistros
sinistros_bimestre = df.groupby('Ano_Bimestre').size().reset_index(name='Quantidade de Sinistros')

# Ordenar os bimestres cronologicamente
sinistros_bimestre = sinistros_bimestre.sort_values(by='Ano_Bimestre')

# Gerar o gráfico de linha
plt.figure(figsize=(10, 6))
plt.plot(sinistros_bimestre['Ano_Bimestre'], sinistros_bimestre['Quantidade de Sinistros'], marker='o')
plt.title('Quantidade de Sinistros ao Longo do Tempo (Agrupado por Bimestres)')
plt.xlabel('Período')
plt.ylabel('Quantidade de Sinistros')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

&ensp;Este gráfico foi elaborado para analisar os sinistros ao longo do período disponível no banco de dados. Observou-se um aumento significativo no quarto bimestre de 2023, seguido por um equilíbrio entre o terceiro bimestre de 2023 e o primeiro bimestre de 2024. Houve um declínio acentuado do quinto para o sexto bimestre de 2023, o que pode indicar um evento específico que afastou os trabalhadores dos serviços de saúde. Este gráfico ajuda a identificar padrões sazonais ou períodos com aumento na utilização dos serviços de saúde. A empresa pode se preparar melhor para esses períodos e otimizar a gestão de recursos. Esses dados podem servir como base para a criação de novos gráficos ou para a formulação de hipóteses futuras.

### Três serviços mais utilizados por titulares em cada mês

In [ ]:
df = df[df['Elegibilidade Sinistro'] == 'TITULAR']

df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format="%d/%m/%Y")

df['Mes'] = df['Dt Data Sinistro'].dt.strftime('%b')  
df['Ano'] = df['Dt Data Sinistro'].dt.year

df['Mes_Ano'] = df['Mes'] + '/' + df['Ano'].astype(str)

ordem_meses = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

df['Mes_Ordem'] = df['Ano'].astype(str) + df['Mes'].apply(lambda x: f'{ordem_meses.index(x):02d}')
df = df.sort_values(by='Mes_Ordem')

freq_servicos_mes_ano = df.groupby(['Mes_Ano', 'Descricao Servico Sinistro']).size().unstack(fill_value=0)

freq_servicos_mes_ano = freq_servicos_mes_ano.loc[(freq_servicos_mes_ano.sum(axis=1) > 0)]

freq_servicos_mes_ano = freq_servicos_mes_ano.reindex(df['Mes_Ano'].unique(), fill_value=0)

top3_servicos_por_mes = freq_servicos_mes_ano.apply(lambda x: x.nlargest(3).index, axis=1)

df_top3 = freq_servicos_mes_ano.apply(lambda x: x[top3_servicos_por_mes.loc[x.name]], axis=1)

cor_servico = {
    'CONSULTA CONSULTORIO (HORARIO NORMAL OU PREESTAB)': '#0000FF', 
    'GLICOSE - PESQUISA E/OU DOSAGEM': '#006400', 
    'HEMOG C/ CONT PLAQ OU FCS (ERITROGR, LEUCOG, PLAQ)': '#FFA500', 
    'PCT (C)OFTM CST,EXM(EXT TMO COER OPTC),MTRS MEDIC ': '#FFFF00', 
    'SESSAO DE PSICOTERAPIA INDIVIDUAL POR PSICOLOGO': '#800080', 
    'CREATININA - PESQUISA E/OU DOSAGEM': '#FFC0CB', 
    'CONSULTA MEDICA POR VIA REMOTA/NAO PRESENCIAL': '#00FF00',
    'MATERIAL': '#FF0000',
    'ATENDIMENTO PRONTO SOCORRO': '#FF00FF',
    'CONSULTA AMBULATORIAL POR NUTRICIONISTA': '#8B4513',
    'DESPESA HOSPITALAR': '#00FFFF',
    'REABILITACAO PERINEAL COM BIOFEEDBACK': '#2F4F4F',
}

cores = [cor_servico.get(service, '#999999') for service in df_top3.columns]

df_top3.plot(kind='bar', stacked=True, figsize=(13, 7), color=cores)

plt.title('Três Serviços Mais Utilizados por Titulares em Cada Mês')
plt.xlabel('Meses/Ano')
plt.ylabel('Frequência de Uso')

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')

plt.grid(axis='y')
plt.show()

&emsp;O gráfico acima exibe quais foram os três serviços mais utilizados por titulares em cada mês que se tem algum registro. Assim, pudemos perceber algumas anomalias, como um aumento significativo de exames oftalmológico durante os meses de setembro e outubro, e alguns fatos, como a constante busca por sessões de psicoterapia e hemogramas. Nesse contexto, o gráfico referido colabora para o desenvolvimento de predições e hipóteses, já que mostra as tendências de aumento ou diminuição da frequência de uso dos principais serviços durante os meses, possibilitando análises por períodos do ano.

### Exploração por feriado

&emsp; Abaixo, foram importados dois arquivos com todos os feriados, que estão em formato JSON, presentes na mesma pasta dos notebooks, sendo eles 'feriados_2023' e 'feriados_2024'. Em seguida, foi adicionada uma coluna que indicava se as datas coincidiam com um dia antes ou depois de feriado, sendo representadas por 'True' ou 'False'.


In [8]:
df["Dt Data Sinistro"] = pd.to_datetime(df["Dt Data Sinistro"])
df["Dia da Semana Sinistro"] = df["Dt Data Sinistro"].dt.day_name("portuguese")

feriados_2023 = pd.read_json('./assets/feriados_2023.json')
feriados_2024 = pd.read_json('./assets/feriados_2024.json')

# Concatena os feriados de 2023 e 2024
feriados = pd.concat([feriados_2023, feriados_2024])

feriados["date"] = pd.to_datetime(feriados["date"])

def verificar_feriado_antes(row):
    data = row["Dt Data Sinistro"]
    sinistro_antes = (feriados["date"] == data + pd.Timedelta(days=1)).any()
    # Verifica se a data é exatamente um dia antes de qualquer feriado
    return sinistro_antes

def verificar_feriado_depois(row):
    data = row["Dt Data Sinistro"]
    sinistro_depois = (feriados["date"] == data - pd.Timedelta(days=1)).any()
    # Verifica se a data é exatamente um dia antes de qualquer feriado
    return sinistro_depois

df["Feriado Antes"] = df.apply(verificar_feriado_antes, axis=1)
df["Feriado Depois"] = df.apply(verificar_feriado_depois, axis=1)

&emsp;Com essas informações, foi elaborado o gráfico a seguir:

In [ ]:
# Calculando quantidade de sinistros nos dias filtrados por feriados anteriormente
valores_feriado_antes = df[df["Feriado Antes"] == True]["Dia da Semana Sinistro"].value_counts().reset_index()
valores_feriado_depois = df[df["Feriado Depois"] == True]["Dia da Semana Sinistro"].value_counts().reset_index()

# Transformando a contagem em um DataFrame
df_valores_antes = pd.DataFrame(valores_feriado_antes)
# Nomeando as colunas
df_valores_antes.columns = ['Dias da Semana', 'Frequencia de Dias Véspera Feriado']

# Transformando a contagem em um DataFrame
df_valores_depois = pd.DataFrame(valores_feriado_depois)
# Nomeando as colunas
df_valores_depois.columns = ['Dias da Semana', 'Frequencia de Dias Pós Feriado']

dff = pd.merge(df_valores_antes, df_valores_depois, on='Dias da Semana', how='left')

# Ordenando as linhas do DataFrame de acordo com a sequência dos dias da semana
ordem_semana = ['Domingo', 'Sábado',  'Sexta-feira', 'Quinta-feira',  'Quarta-feira', 'Terça-feira', 'Segunda-feira']
# ordem_semana = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo']
dff['Dias da Semana'] = pd.Categorical(dff['Dias da Semana'], categories=ordem_semana, ordered=True)
dff = dff.sort_values('Dias da Semana')

# Definindo o deslocamento entre as barras
largura = 0.4  # Largura das barras (ajuste conforme necessário)

# Definindo as posições das barras
posicao_barras = np.arange(len(dff['Dias da Semana']))

# Criando o gráfico de barras horizontais lado a lado
plt.barh(posicao_barras + largura/2, dff['Frequencia de Dias Véspera Feriado'], 
         height=largura, label='Véspera de Feriado', zorder=2)
plt.barh(posicao_barras - largura/2, dff['Frequencia de Dias Pós Feriado'], 
         height=largura, label='Pós Feriado', zorder=2)


plt.grid(axis='x', zorder=1)
# Colocando as legendas e o título
plt.title('Número de sinistros registrados na véspera ou um dia após um feriado')
plt.xlabel('Número de registros')
plt.ylabel('Dia da semana com ocorrência de feriado')

# Ajustando os rótulos do eixo Y para exibir os dias da semana
plt.yticks(posicao_barras, dff['Dias da Semana'])

# Adicionando a legenda
plt.legend()

plt.show()

&emsp;Nele, é possível analisar a incidência de uso do plano de saúde nas vésperas e um dia após os feriados. Nesse contexto, é notável a quantidade de registros após feriados em sextas-feiras, ou antes de quartas-feiras e terças-feiras. Além disso, pode-se observar uma baixa frequência de dias associados a datas comemorativas em domingos.

## Hipóteses

&ensp; A partir dos gráficos acima, foi possível analisar alguns dados que apresentavam uma frequência de uso relevante. Com base nisso, levantamos algumas hipóteses, que podem ou não estar corretas. Vale ressaltar que apresentaremos, a seguir, algumas informações que reforçam essas hipóteses e fortalecem nosso embasamento para suas criações.   

##### 1º O alto índice de psicoterapia dentro da empresa indica uma crescente demanda de serviços relacionados a saúde mental.

&ensp; Após analisar os 10 serviços mais utilizados no plano de saúde e os 5 maiores gastos totais, foi possível observar uma relevância nos casos de psicoterapia, o que evidencia uma alta demanda por esses tipos de serviço. Para aprofundar essa pesquisa, buscamos entender a idade das pessoas que utilizam esse serviço, assim como o gênero e em qual das sedes elas trabalham.

#### Tabela quantidade por gênero
&ensp;Relação de casos por gênero:

In [ ]:
filtro = (df["Descricao Servico Sinistro"] == "SESSAO DE PSICOTERAPIA INDIVIDUAL POR PSICOLOGO")
df_psicologia = df[filtro]
df_psicologia["Sexo Sinistro"].value_counts()

&ensp; A partir desses dados, foi possível identificar que a maioria dos usuários desse tipo de serviço são mulheres, mesmo sendo a minoria em número de colaboradores na empresa. Uma hipótese para isso é que as mulheres cuidam mais da saúde do que os homens. Isso sugere a necessidade de criar programas de conscientização para aumentar o uso desses serviços, que contribuem para a manutenção de uma boa qualidade de vida.

#### Tabela de faixa-etária de idade
&ensp; Agora, pensando na idade, foi realizado um levantamento do número de usos por faixa etária, conforme a tabela a seguir:

In [ ]:
filtro2 = (
    (df["Descricao Servico Sinistro"] == "SESSAO DE PSICOTERAPIA INDIVIDUAL POR PSICOLOGO") 
    
)

df_psicologia_feminino = df[filtro2]
df_psicologia_feminino["Faixa-Etária Nova Sinistro"].value_counts()

&ensp; A partir das informações exibidas acima, foi possível visualizar o número de casos por faixa etária. Observa-se uma tendência de uso mais frequente entre pessoas de 29 a 38 anos, além de um uso reduzido por parte das pessoas com 49 anos ou mais. Isso leva à reflexão de que as pessoas de 29 a 38 anos cuidam mais da saúde, o que é um ponto positivo. Por outro lado, elas podem estar mais suscetíveis a problemas relacionados à saúde mental, possivelmente devido à autocobrança ou outros fatores pessoais que podem contribuir para esses problemas nessa idade. Seguindo a mesma linha de raciocínio, mas agora voltando-se para o público mais velho, é possível considerar possíveis causas para essas estimativas quantitativas. Uma delas é o grau de amadurecimento dessas pessoas, que pode ser elevado, levando a menos problemas relacionados à saúde mental. Outra possibilidade é que essas pessoas tenham receio ou uma certa resistência ao uso desses serviços.

#### Tabela Elegibilidade Sinistro

&ensp; Para identificar quem são os usuários desse sinistro, foi criada a tabela a seguir, que visa quantificar a elegibilidade do sinistro:

In [ ]:
df_psicologia["Elegibilidade Sinistro"].value_counts()

&ensp; A partir desse levantamento, foi possível verificar que a maior parte do uso é feita pelos dependentes, o que evidencia uma baixa adesão entre os próprios colaboradores. Isso pode reforçar a ideia de que deve haver campanhas de conscientização direcionadas aos colaboradores, explicando a importância desse serviço e como ele pode impactar positivamente no bem-estar e até mesmo a produtividade.

### 2º A frequência de exames de colesterol é influenciada pela faixa etária, com idosos realizando significativamente mais exames.

&ensp; Foi observado o número de alguns exames relacionados ao colesterol no gráfico dos "top 10" exames mais utilizados. No entanto, além dos exibidos no gráfico, havia outros exames com um número menor de usos, mas que também contribuem para o total. Portanto, foi criada a tabela a seguir, que mostra a relação de cada exame específico com o número de casos. Após visualizar esses números, somamos todos para obter a quantidade total de serviços utilizados relacionados ao colesterol.

In [ ]:

filtro_colesterol = (
    (df["Descricao Servico Sinistro"] == "COLESTEROL TOTAL - PESQUISA E/OU DOSAGEM") |
    (df["Descricao Servico Sinistro"] == "COLESTEROL (HDL) - PESQUISA E/OU DOSAGEM") |
    (df["Descricao Servico Sinistro"] == "COLESTEROL (LDL) - PESQUISA E/OU DOSAGEM") |
    (df["Descricao Servico Sinistro"] == "COLESTEROL (VLDL) - PESQUISA E/OU DOSAGEM")
)

df_colesterol = df[filtro_colesterol]
tabela_colesterol = df_colesterol["Descricao Servico Sinistro"].value_counts()
print(tabela_colesterol)
total_colesterol = filtro_colesterol.sum()
print(f"NÚMERO TOTAL DE EXAMES RELACIONADOS AO COLESTEROL: {total_colesterol} EXAMES" )

&ensp; A partir desses dados, é possível visualizar um alto índice de uso, o que gera uma reflexão sobre a saúde dos colaboradores, que podem estar com colesterol alto ou próximo desse nível. Por outro lado, é possível que eles estejam apenas preocupados com a saúde e realizem exames de rotina para monitorar seu estado de saúde.

### Tabela quantidade de exames de colesterol por gênero

&ensp; Relação de quantidade de casos por gênero:

In [ ]:
contagem_por_genero = df_colesterol["Sexo Sinistro"].value_counts()


print("Contagem de exames de colesterol por gênero:")
print(contagem_por_genero)


&ensp; A partir desses dados, foi possível observar que as mulheres realizam mais exames do que os homens, mesmo sendo minoria em termos de quantidade dentro da empresa. Assim, algumas sub-hipóteses podem ser levantadas: as mulheres na Unipar podem ter um maior índice de casos de colesterol alto, ou elas podem se prevenir e cuidar mais da saúde do que os homens. No entanto, é importante ressaltar que os homens também representam um número significativo de casos.

#### Tabela de faixa-etária
&ensp; Agora pensando na idade, foi feito o levantamento do número de uso por faixa-etária de idade, a partir da seguinte tabela:

In [ ]:
contagem_por_faixa_etaria = df_colesterol["Faixa-Etária Nova Sinistro"].value_counts()

print("Número de casos de colesterol por faixa etária:")
print(contagem_por_faixa_etaria)


&ensp; A partir desses dados, foi possível visualizar um grande número de casos entre pessoas de 0 a 18 anos, que são, em sua maioria, dependentes. Portanto, esses dados foram eliminados, pois não são o foco do nosso estudo. Observando os outros dados, notamos que pessoas com 59 anos ou mais apresentam um índice elevado de casos. Assim, é possível propor a hipótese de que boa parte dessas pessoas tende a precisar de cuidados maiores e pode ter níveis elevados de colesterol, o que representa um risco para a saúde. Além disso, há uma média de 500 exames entre pessoas nas faixas etárias de 34 a 58 anos. Isso sugere que esses grupos podem estar suscetíveis a níveis elevados de colesterol, devido a falta de cuidado ou que estão mais preocupados com sua saúde.

#### Tabela Elegibilidade Sinistro

&ensp; Em busca de encontrar quem são os usuários desse sinistro, foi feita a seguinte tabela que visa quantificar a elegibilidade do sinistro:

In [ ]:

contagem_por_sede = df_colesterol["Elegibilidade Sinistro"].value_counts()

print("Número de casos de colesterol por sede da empresa:")
print(contagem_por_sede)


&ensp; A partir dos dados acima, é possível relacionar o número de casos com a elegibilidade do sinistro. Neste caso, observa-se que a maioria dos casos são de dependentes, mas o titular, ou seja, quem está trabalhando na empresa e para quem os programas serão voltados, também apresenta um número significativo de casos.

&ensp; Com base nas informações, é possível destacar que pessoas acima de 59 anos podem estar mais suscetíveis ao colesterol alto. Além disso, há um número significativo de pessoas entre 34 e 58 anos que pode apresentar colesterol alto em um futuro próximo. Portanto, é interessante propor programas de prevenção e cuidados para esses grupos, especialmente em casos mais avançados.

&ensp; Ainda nessa linha de raciocínio, é importante considerar a questão de gênero, uma vez que as mulheres podem estar se cuidando e se prevenindo mais ou podem ser mais suscetíveis a esse problema. Por outro lado, o menor número de casos entre homens pode estar relacionado com um cuidado menor com a saúde, o que pode levar a surpresas no futuro, uma vez que não estariam monitorando adequadamente seus níveis de colesterol.

### 3° Alto número de exames de glicose indicam um mapa de risco relacionado à diabetes.

&emsp;Também examinamos a importância dos exames de glicose, que figuram como o quarto serviço mais solicitado e o quinto em termos de valor total pago. O objetivo é determinar se o elevado número desses exames pode sinalizar um possível padrão de risco associado ao diabetes. Realizamos uma investigação mais detalhada para entender o perfil dos pacientes que procuram esse exame.

In [ ]:
filtro_glicose = (df['Descricao Servico Sinistro'] == "GLICOSE - PESQUISA E/OU DOSAGEM")
df_glicose = df[filtro_glicose]

total_glicose = filtro_glicose.sum()
print(f"NÚMERO TOTAL DE EXAMES RELACIONADOS A GLICOSE: {total_glicose} EXAMES" )

&emsp;Esses dados indicam um elevado uso, sugerindo uma preocupação sobre a saúde dos funcionários, que podem estar com a glicose elevada ou em risco de atingi-la. Por outro lado, é possível que esses funcionários estejam apenas adotando uma postura preventiva.

#### Consultas de glicose por gênero

&emsp;Em uma análise inicial, torna-se essencial identificar o perfil dos indivíduos que realizam exames de glicose com frequência. A primeira avaliação concentrou-se no gênero, examinando a proporção entre homens e mulheres que realizam esses exames e as implicações que isso traz para a pesquisa.

In [ ]:
filtro_glicose = (df['Descricao Servico Sinistro'] == "GLICOSE - PESQUISA E/OU DOSAGEM")
df_glicose = df[filtro_glicose]

df_glicose["Sexo Sinistro"].value_counts()

&emsp;Dessa forma, observa-se que as mulheres realizam um número significativamente maior de exames de glicose em comparação aos homens, o que é notável, considerando que a empresa possui uma maior proporção de homens em relação às mulheres.

#### Consultas de glicose por faixa etária

&emsp;Em uma segunda análise, avaliou-se a faixa etária dos indivíduos que realizam exames de glicose com frequência, com o objetivo de identificar quais grupos etários são mais propensos ao acompanhamento preventivo.

In [ ]:
filtro_glicose = (df['Descricao Servico Sinistro'] == "GLICOSE - PESQUISA E/OU DOSAGEM")
df_glicose = df[filtro_glicose]

df_glicose["Faixa-Etária Nova Sinistro"].value_counts()

&emsp;Com base na análise, o grupo mais presente é o de indivíduos com 59 anos ou mais, seguido pelo grupo de 0 a 18 anos (provavelmente filhos dos colaboradores). Essa análise possibilita compreender melhor o perfil etário dos participantes dos exames de glicose e auxilia na identificação de padrões de comportamento em relação ao cuidado com a saúde, contribuindo para a formulação de estratégias de prevenção e promoção de saúde mais eficazes, direcionadas às faixas etárias que demonstram maior engajamento.

#### Consultas de glicose por elegibilidade

&emsp;Por fim, avaliou-se a elegibilidade desses indivíduos para identificar a proporção entre titulares, dependentes e agregados.

In [ ]:
filtro_glicose = (df['Descricao Servico Sinistro'] == "GLICOSE - PESQUISA E/OU DOSAGEM")
df_glicose = df[filtro_glicose]

df_glicose["Elegibilidade Sinistro"].value_counts()

&emsp;O estudo constatou um maior número de dependentes em relação aos titulares, indicando que esses exames são mais frequentemente realizados por filhos e cônjuges dos colaboradores. Essa análise possibilita compreender a distribuição dos exames de glicose entre diferentes categorias de beneficiários, fornecendo insights valiosos para a elaboração de políticas de saúde mais inclusivas e personalizadas, que atendam de forma adequada às necessidades de cada grupo e promovam o bem-estar geral da população coberta.

&emsp;Em conclusão, não é possível, por enquanto, determinar se o alto índice de exames de glicose indica um mapa de risco associado ao diabetes, devido às informações limitadas disponíveis. No entanto, identificar o perfil daqueles que realizam esses exames é fundamental, pois permite uma melhor compreensão das necessidades e dos comportamentos de saúde dos diferentes grupos de beneficiários, possibilitando a implementação de estratégias preventivas mais direcionadas e eficazes, além de promover um acompanhamento mais adequado para a prevenção de doenças como o diabetes.